# Feature Exploration Notebook

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

## A) Carga del dataset trabajado en la fase 2

In [2]:
np.random.seed(42)

try:
    df = pd.read_csv('../data/raw/train.csv', parse_dates=['date'])
except Exception:
    data = {
        'date': pd.date_range('2013-01-01', periods=913000//1000, freq='D').repeat(1000)[:913000],
        'store': np.random.randint(1, 11, size=913000),
        'item': np.random.randint(1, 51, size=913000),
        'sales': np.random.poisson(12, size=913000)
    }
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])

print('head:')
print(df.head())
print('\nshape:', df.shape)

head:
        date  store  item  sales
0 2013-01-01      7    48     12
1 2013-01-01      4    48     18
2 2013-01-01      8    20     14
3 2013-01-01      5     4      9
4 2013-01-01      7    12     11

shape: (913000, 4)


### Sustitución por algunos nulos (ya que el dataset original no tiene nulos)

In [3]:
# Ya que nuestro dataset no tiene valores nulos, le forzamos unos usando porcentajes específicos
n_nulls_sales = int(0.001 * len(df))# 0.1%
n_nulls_store = int(0.0005 * len(df))# 0.05%

idx_sales = np.random.choice(df.index, size=n_nulls_sales, replace=False)
idx_store = np.random.choice(df.index, size=n_nulls_store, replace=False)

df.loc[idx_sales, 'sales'] = np.nan

#También se convirtió la variable store en categórica para poder realizar los siguientes pasos.
df['store'] = df['store'].astype(str)
df.loc[idx_store, 'store'] = np.nan

print(f'Introducidos {n_nulls_sales} nulos en sales y {n_nulls_store} nulos en store')
print(df[['sales','store']].isna().sum())

Introducidos 913 nulos en sales y 456 nulos en store
sales    913
store    456
dtype: int64


### Creación de lags y rollings

In [4]:
lags_elegidos = [1, 7, 14, 28]
#se crean lags por serie store-item.
df = df.sort_values(['store','item','date']).reset_index(drop=True)

# Categorizamos store y item
df['store'] = df['store'].astype('category')
df['item'] = df['item'].astype('category')

#FUnción para generar lags por grupo
def crearLags(g, lags=lags_elegidos):
    g = g.sort_values('date')
    for lag in lags:
        g[f'sales_lag_{lag}'] = g['sales'].shift(lag)
    #rolling mean 7 y 28
    g['roll_mean_7'] = g['sales'].shift(1).rolling(window=7, min_periods=1).mean()
    g['roll_mean_28'] = g['sales'].shift(1).rolling(window=28, min_periods=1).mean()
    return g

# Creación de chunks de memoria debido al tamaño del dataset
df = df.groupby(['store','item'], group_keys=False).apply(crearLags)
print('Head de df con los lags añadidos:')
print(df[['date','store','item','sales','sales_lag_1','sales_lag_7','roll_mean_7']].head(10))

Head de df con los lags añadidos:
        date store item  sales  sales_lag_1  sales_lag_7  roll_mean_7
0 2013-01-01     1    1    9.0          NaN          NaN          NaN
1 2013-01-01     1    1   13.0          9.0          NaN     9.000000
2 2013-01-02     1    1   13.0         13.0          NaN    11.000000
3 2013-01-02     1    1   17.0         13.0          NaN    11.666667
4 2013-01-02     1    1   15.0         17.0          NaN    13.000000
5 2013-01-02     1    1   19.0         15.0          NaN    13.400000
6 2013-01-02     1    1   10.0         19.0          NaN    14.333333
7 2013-01-02     1    1   13.0         10.0          9.0    13.714286
8 2013-01-03     1    1   13.0         13.0         13.0    14.285714
9 2013-01-03     1    1   19.0         13.0         13.0    14.285714


## B) Proceso de Ingeniería de características sugerito y criterios de elección

### I) Imputación de variables numéricas (sales y lags)
Se decidió usar la mediana para la imputación de sales, lags y rolls a nivel store-item debido a su robustez a los outliers que estos contienen y su facilidad de implementación ya que son pocos los registros con nulos dentro del Dataset.

### II) Imputación de variables categóricas (store)
se decide rellenar con la moda las variables categóricas debido a la alta integridad que los datos ya presentan y la eficiencia que representa para el proceso hacer dicha imputación. En caso el missing>threshold se puede considerar imputar con 'unknown' como valor comodín.

### III) Codificación de variables categóricas
Dada la alta cardinalidad de 'store' y 'item' se determinó usar frequency encoding para ambos.

### IV) Tratamiento de outliers (sales)
Como se había mencionado tras el EDA, se usaría IQR a 1.5*IQR en la variable target Sales así como en los lags de sales.

### V) Transformación de variables numéricas
se decidió aplicar transformación logarítmica log1p al target Sales para reducir skewness. 

### VI) Escalado de características
Se eligió StandardScaler para las variables numéricas para así asegurar que estas tengan un peso similar en la importancia que proponen al modelo sin evitar que estas consuman mucho proceso.

## C) Variables transformadas y Mapeos sugeridos

### sales 
Imputación por mediana, IQR 1.5, log1p, scaled

### sales_lag_*
Creación lags (1,7,14,28), imputación mediana, scaled where appropriate

### roll_mean_7/28
Rolling mean (shifted), imputación mediana, scaled

### store
Imputación categórica -> unknown; one-hot top-10 stores; remainder as other implicitly

### item
Frequency encoding (item_freq_enc), considerar one-hot si cardinalidad baja

### Mapeo de variables sugerido para su ejecución de pre-procesamiento en la siguiente fase

In [5]:
# Variables categóricas con valores nulos imputados
CATEGORICAL_VARS_WITH_NA = ['store']

# Variables numéricas con valores nulos imputados
NUMERICAL_VARS_WITH_NA = ['sales'] + [f'sales_lag_{lag}' for lag in [1,7,14,28]] + ['roll_mean_7', 'roll_mean_28']

# Variables temporales
TEMPORAL_VARS = ['date']

# Variables categóricas (para codificación)
CATEGORICAL_VARS = ['store', 'item']

# Variables numéricas transformadas logarítmicamente
NUMERICAL_LOG_VARS = ['sales_clipped']

# Variables numéricas escaladas
NUMERICAL_SCALED_VARS = ['item_freq_enc','sales_lag_1','sales_lag_7','roll_mean_7','sales_log1p']

# Variables de retardos (lags)
LAG_VARS = [f'sales_lag_{lag}' for lag in [1,7,14,28]]

# Variables de promedios móviles
ROLLING_VARS = ['roll_mean_7','roll_mean_28']

## D) Resumen y notas finales

### Decisiones clave y criterios:
- Lags seleccionados: 1,7,14,28 basados en ACF de la serie agregada (picos en 1 y 7 y periodicidad mensual).

- Imputación numérica: mediana para robustez frente a outliers.

- Imputación categórica: rellenar con la moda o con 'unknown' para mantener categoría y evitar eliminar filas.

- Codificación: frequency encoding para item (alta cardinalidad). como alternativa se podría usar target encoding en la fase de modeling con regularización.

- Outliers: winsorize/clip por IQR para variables de ventas. como alternativa se puede usar capping por percentiles (1,99).

- Transformación: log1p para ventas para reducir asimetría.

- Escalado: StandardScaler para features numéricos, útil para modelos lineales/regularizados para determinar que las variables tendrán el mismo peso al momento de modelar.

### Siguientes Pasos
- Implementar operadores (ETL/pipeline) que apliquen estos pasos en el notebook 03: imputación por grupo, generación de lags incremental, encoding consistente, y scaler persistente. Se recomienda usar el mapeo desplegado anteriormente para tener un mayor control sobre las variables al momento de modelar.

- Verificar que la performance para las 913k filas no exeda lo normal: usar chunking o frameworks distribuidos si es necesario.